<a href="https://colab.research.google.com/github/Xornotor/PPGEEC-CompEvolutiva-Atividades/blob/main/Avalia%C3%A7%C3%A3o_1A_Algoritmos_Gen%C3%A9ticos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




# **Avaliação 1A - Computação Evolutiva e Meta-heurísticas**

**Docente:** Prof. Dr. Edmar Egídio Purcino de Souza

**Discentes:** André Paiva, Gabriel Lucas, Márcio Barros e Shaísta Câmara

## 1 - Importação de Dependências e Parametrização

In [ ]:
import random
import numpy as np
from matplotlib import pyplot as plt
from plotly import graph_objects as go
from itertools import combinations
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150

In [ ]:
# Parâmetros do algoritmo genético

#----------------------------------------------------------------
# Escolha a função de fitness a ser usada: "dropwave" ou "levi"
FUNCTION_NAME = "dropwave"
#FUNCTION_NAME = "levi"
#----------------------------------------------------------------

POPULATION_SIZE = 100
GENERATION_COUNT = 100
CROSSOVER_RATE = 0.8
MUTATION_RATE = 0.5
CHROMOSOME_LENGTH = 2
LOWER_BOUND = -100
UPPER_BOUND = 100

## 2 - Definição de função objetivo (*fitness*)


In [ ]:
if FUNCTION_NAME == "dropwave":
    def fitness(chromosome):
        # Dropwave
        x1 = chromosome[0]
        x2 = chromosome[1]
        r = np.sqrt(x1**2 + x2**2)
        numerator = 1 + np.cos(12 * r)
        denominator = 0.5 * (x1**2 + x2**2) + 2
        return - numerator / denominator
elif FUNCTION_NAME == "levi":
    def fitness(chromosome):
        # Levi n. 13
        x1 = chromosome[0]
        x2 = chromosome[1]
        term1 = np.sin(3 * np.pi * x1)**2
        term2 = (x1 - 1)**2 * (1 + np.sin(3 * np.pi * x2)**2)
        term3 = (x2 - 1)**2 * (1 + np.sin(2 * np.pi * x2)**2)
        return term1 + term2 + term3
else:
    raise ValueError("Função de fitness desconhecida.")

## 3 - Plot da função objetivo

In [ ]:
if FUNCTION_NAME == "dropwave":
    x = y = np.linspace(-5, 5, 500)
elif FUNCTION_NAME == "levi":
    x = y = np.linspace(-10, 10, 500)

chromosome = np.meshgrid(x, y)
Z = fitness(chromosome)

fig = go.Figure(data=[go.Surface(z=Z, x=x, y=y, colorscale='rainbow')])
fig.update_layout(title=f"Função {FUNCTION_NAME.capitalize()}",
                  autosize=False, width=800, height=600,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [ ]:
# Generate a random chromosome
# só a parte dos genes, entre os limites determinados
def generate_chromosome():
    return [random.uniform(LOWER_BOUND, UPPER_BOUND) for _ in range(CHROMOSOME_LENGTH)]

# Perform selection using tournament selection
#faz um sorteio do de melhor fitness (valor da função f(x1,x2)) entre 10 individuos aletorios
#nesse caso como o for é igual a população, toda a população é "substituida" dessa forma
#como é um problema de minimização o winner é o valor minimo (menor fitness)
# salva tanto o melhor de toda a população, como o melhor e cada torneio, garantindo que o melhor nao vai ser perdido
def selection(population):
    tournament_size = 10
    selected_parents = []
    for _ in range(len(population)):
        tournament = random.sample(population, tournament_size-1)
        winner = min(tournament, key=lambda x: x[1]) # x[1] é o fitness na populatin/tourment
        selected_parents.append(winner[0]) # winner[0] pega somente os valores dos genes (x1,x2)
    best_individual = min(population, key=lambda x: x[1])[0]
    selected_parents.append(best_individual)
    return selected_parents

# Perform crossover between two parents
#define aletoriamente um ponto de corte no cromossomo e depoia, corta os dois pais nesse ponto e junta de foma cruzana nos filhos, ou seja, parte 1 do pai +pare 2 da mae e parte 2 do pais + parte 2 da mae

def crossover(parent1, parent2):
    if random.random() < CROSSOVER_RATE: #random.random() retorna entre 0 e 1
        crossover_point = random.randint(1, CHROMOSOME_LENGTH - 1) # escolhe um ponto para corte do gene, porem como sao 2 genes sempre é o mesmo ponto para as nossas funções
        child1 = parent1[:crossover_point] + parent2[crossover_point:]
        child2 = parent2[:crossover_point] + parent1[crossover_point:]
        return child1, child2
    else:
        return parent1, parent2

# Perform mutation on an individual
#adciona uma mutação (valor) a cada gene (x1 ou x2)entre -1 e 1, para ca individuo (entrada (x1,x2))
def mutate(individual):
    mutated_individual = individual.copy()
    for i in range(CHROMOSOME_LENGTH):
        if random.random() < MUTATION_RATE:
            mutated_individual[i] += random.uniform(-1, 1)
    return mutated_individual

def calculate_diversity_metrics(population):
    # Extrai os cromossomos
    genes = np.array([chromosome for chromosome, _ in population])

    # Desvio padrão
    std_devs = np.std(genes, axis=0)

    # Distância média ao centro
    mean_distance_to_center = np.mean(np.linalg.norm(genes - np.mean(genes, axis=0), axis=1))

    # medida espacial
    distances = [np.linalg.norm(a - b) for a, b in combinations(genes, 2)]
    mean_pairwise_distance = np.mean(distances)

    return std_devs, mean_distance_to_center, mean_pairwise_distance

# Generate an initial population and evaluate
population = [(generate_chromosome(), 0) for _ in range(POPULATION_SIZE)]
population = [(ind, fitness(ind)) for ind, _ in population]

In [ ]:
# Genetic Algorithm main loop

bestValue = []

#Variaveis para calculo de diversidade
std_devs_list = []
mean_distance_to_center_list = []
mean_pairwise_distance_list = []

for idx in range(GENERATION_COUNT):
    # Avalia a população com a função de fitness escolhida
    #ele sempre corta os ultimos individuos nessa linha, pq no final da population size  +elite
    population = [(chromosome, fitness(chromosome)) for chromosome, _ in population]

    # Seleciona os pais
    parents = selection(population) # retorna um vetor de size population+1, isso é importante pro loop seguinte funcionar, como foi feito

    # Realiza o crossover e a mutação para gerar descendentes
    offspring = []
    #vai de 2 em 2 dentro da população
    for i in range(0, POPULATION_SIZE, 2):
        parent1 = parents[i]
        parent2 = parents[i + 1]
        child1, child2 = crossover(parent1, parent2)
        child1 = mutate(child1)
        child2 = mutate(child2)
        offspring.extend([child1, child2])

    # Recria a população a partir dos descendentes
    # 0 é um valore temporario de fitness
    population_x = [(chromosome, 0) for chromosome in offspring]

    # Elitismo: mantém os melhores indivíduos da geração anterior
    # a função sorted organiza por ordem de fitness por isso que elecionar o vetor até num_elites funciona
    num_elites = 10
    elites = sorted(population, key=lambda x: x[1])[:num_elites]

    best_individual = min(population, key=lambda x: x[1])[0] # o [0] no final é para pegar apenas o cromossomo, depois de avaliar o menor fitness da população
    population = elites + population_x

    # Reavalia o melhor indivíduo para registrar
    fitness_value = fitness(best_individual)
    bestValue.append(fitness_value)

    # Calcula diversidade da população atual
    std_devs, dist_to_center, pairwise_dist = calculate_diversity_metrics(population)

    # Armazena os valores
    std_devs_list.append(std_devs)
    mean_distance_to_center_list.append(dist_to_center)
    mean_pairwise_distance_list.append(pairwise_dist)

    print("Generation: ", idx,
          "| Best Fitness: {:.6f}".format(fitness_value),
          "| Best Solution: ", best_individual)

In [ ]:
x = np.arange(0, len(bestValue))
y = bestValue

fig = plt.figure()
ax = plt.gca()

plt.plot(x, y, color='Plum', linestyle='--', linewidth=2,
         marker='o', markeredgewidth=1, markerfacecolor='FireBrick',
         markeredgecolor='black', markersize=2)

plt.xlabel('Generations')
plt.ylabel('Fitness Function')

# Mostra no título qual função está sendo usada
plt.title(f'Convergence Curve - {FUNCTION_NAME.capitalize()} Function')

if(FUNCTION_NAME == "dropwave"):
    plt.axhline(y = -1, color ="red", linestyle ="-", zorder=1.5, linewidth=2)
elif(FUNCTION_NAME == "levi"):
    plt.axhline(y = 0, color ="red", linestyle ="-", zorder=1.5, linewidth=2)

plt.grid(True)
plt.show()
plt.close(fig)

# Identificar última mudança significativa (> 1e-4) em bestValue
threshold = 1e-4
last_significant_idx = 0

for i in range(1, len(bestValue)):
    if abs(bestValue[i] - bestValue[i - 1]) > threshold:
        last_significant_idx = i

print(f"Última mudança significativa (>1e-4) ocorreu na Geração de Convergência: {last_significant_idx}")
print(f"Best Fitness : {bestValue[last_significant_idx]:.6f}")
print(f"POPULATION_SIZE: {POPULATION_SIZE}")
print(f"GENERATION_COUNT: {GENERATION_COUNT}")
print(f"CROSSOVER_RATE: {CROSSOVER_RATE}")
print(f"MUTATION_RATE: {MUTATION_RATE}")
print(f"CHROMOSOME_LENGTH: {CHROMOSOME_LENGTH}")
print(f"LOWER_BOUND: {LOWER_BOUND}")
print(f"UPPER_BOUND: {UPPER_BOUND}")

## x - Plot das medidas de diversidade

In [ ]:
def plot_diversity_metrics(std_devs_list, mean_distance_to_center_list, mean_pairwise_distance_list):
    generations = range(len(std_devs_list))
    std_x1 = [std[0] for std in std_devs_list]
    std_x2 = [std[1] for std in std_devs_list]

    plt.figure(figsize=(10, 3))

    # Gráfico 1: Desvio padrão dos genes
    plt.subplot(1, 3, 1)
    plt.plot(generations, std_x1, label='Std x1')
    plt.plot(generations, std_x2, label='Std x2')
    plt.title('Desvio Padrão dos Genes')
    plt.xlabel('Geração')
    plt.ylabel('Desvio Padrão')
    plt.legend()
    plt.grid(True)

    # Gráfico 2: Distância ao centro
    plt.subplot(1, 3, 2)
    plt.plot(generations, mean_distance_to_center_list, color='orange')
    plt.title('Distância Média ao Centro')
    plt.xlabel('Geração')
    plt.ylabel('Distância')
    plt.grid(True)

    # Gráfico 3: Distância média entre pares
    plt.subplot(1, 3, 3)
    plt.plot(generations, mean_pairwise_distance_list, color='green')
    plt.title('Distância Média entre Indivíduos')
    plt.xlabel('Geração')
    plt.ylabel('Distância')
    plt.grid(True)

    plt.tight_layout()
    plt.show()

plot_diversity_metrics(std_devs_list, mean_distance_to_center_list, mean_pairwise_distance_list)